In [303]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import tiktoken
import math
import random

In [245]:
EMBEDDING_DIM = 256
BATCH_SIZE = 8
CONTEXT_LEN = 128
TEXT_FILE = "./datasets/tinyshakespeare.txt"

In [246]:
tokenizer = tiktoken.encoding_for_model("gpt2")

In [247]:
with open(TEXT_FILE, "r") as f:
    data = f.read() 

In [248]:
tokenized_data = torch.tensor(tokenizer.encode(data))

In [250]:
# partitioning scheme from 
# https://d2l.ai/chapter_recurrent-neural-networks/language-model.html#partitioning-sequences
d = torch.randint(CONTEXT_LEN, size = (1,))[0].item()

# discard last item which may be of diff size
token_partitions_X = torch.stack(torch.split(tokenized_data[d:], CONTEXT_LEN)[:-1])
token_partitions_Y = torch.stack(torch.split(tokenized_data[d+1:], CONTEXT_LEN)[:-1])

In [252]:
sample_batch = token_partitions_X[:BATCH_SIZE]

embeddings

In [ ]:
# TODO do we need a max_norm? seems like this would be important
# depending on positional embedding scheme
embedder = nn.Embedding(
    num_embeddings = tokenizer.n_vocab,
    embedding_dim = EMBEDDING_DIM
)
# use learned positional embeddings for simplicity
# TODO what are the tradeoffs with fixed positional embeddings besides less storage?
positional_embedder = nn.Embedding(
    num_embeddings = CONTEXT_LEN,
    embedding_dim = EMBEDDING_DIM
)
context_idx_tensor = torch.tensor(list(range(CONTEXT_LEN)))
positional_embeddings = positional_embedder(context_idx_tensor)
token_embeddings = embedder(sample_batch)
embeddings = token_embeddings + positional_embeddings

attention

In [153]:
class Attention(nn.Module):

    def __init__(self, dropout = 0.5):
        super().__init__()
        self.softmax = nn.Softmax(dim=-1)
        self.dropout = nn.Dropout(dropout)

    def _masked_softmax(self, logits, mask):
        logits.masked_fill_(mask == 0, float('-inf'))
        return self.softmax(logits)

    def forward(self, queries, keys, values):
        assert queries.shape == keys.shape
        assert queries.shape[1] == values.shape[1]

        ctx_len = queries.shape[1]
        qk_dim = queries.shape[2]

        # transpose (ctx, embedding) dims
        scaled_dot_prod = torch.bmm(queries, keys.transpose(1, 2)) / math.sqrt(qk_dim)

        # TODO autoregressive mask; generalize this
        mask = torch.tril(torch.ones(ctx_len, ctx_len))  

        attention_weights = self._masked_softmax(scaled_dot_prod, mask)

        # The book uses dropout for weights but doesn't explain why that specifically?
        # https://d2l.ai/chapter_attention-mechanisms-and-transformers/attention-scoring-functions.html#scaled-dot-product-attention
        return torch.bmm(self.dropout(attention_weights), values)


In [186]:
class MultiHeadAttention(nn.Module):

    def __init__(self, num_heads, qkv_dim, dropout = 0.5):
        super().__init__()

        assert qkv_dim % num_heads == 0

        self.qkv_dim = qkv_dim
        self.num_heads = num_heads
        self.dropout = dropout

        self.attention_heads = [
            Attention(dropout = dropout)
            for _ in range(num_heads)
        ]

        self.q_linear = nn.LazyLinear(qkv_dim)
        self.k_linear = nn.LazyLinear(qkv_dim)
        self.v_linear = nn.LazyLinear(qkv_dim)
        self.output_layer = nn.LazyLinear(qkv_dim)

    def forward(self, queries, keys, values):
        assert queries.shape == keys.shape
        assert queries.shape == values.shape
        assert queries.shape[-1] == self.qkv_dim

        head_dim = self.qkv_dim // self.num_heads
        # TODO these are just truncated in the book?!
        # https://d2l.ai/chapter_attention-mechanisms-and-transformers/multihead-attention.html#implementation
        # why not use a linear layer to learn a lower dim value?
        pq = torch.split(self.q_linear(queries), head_dim, dim=-1)
        pk = torch.split(self.k_linear(keys), head_dim, dim=-1)
        pv = torch.split(self.v_linear(values), head_dim, dim=-1)

        # TODO use transpose trick to parallelize across heads
        head_vals = [
            self.attention_heads[i](pq[i], pk[i], pv[i])
            for i in range(self.num_heads)
        ]

        cat_head_vals = torch.cat(head_vals, dim=-1)
        return self.output_layer(cat_head_vals)

In [200]:
class PositionwiseFFN(nn.Sequential):
    def __init__(self, ffn_hidden_dim: int, ffn_output_dim: int):
        super().__init__(
            nn.LazyLinear(ffn_hidden_dim),
            nn.ReLU(),
            nn.LazyLinear(ffn_output_dim)
        )

In [207]:
class AddNorm(nn.Module):

    def __init__(self, norm_shape, dropout = 0.5):
        super().__init__()

        self.dropout = nn.Dropout(dropout)
        self.ln = nn.LayerNorm(norm_shape)

    def forward(self, X, Y):
        return self.ln(X + self.dropout(Y))

In [208]:
class TransformerDecoderBlock(nn.Module):

    def __init__(self, num_heads: int, model_dim: int, ffn_hidden_dim: int, dropout = 0.5):
        super().__init__()

        self.attention = MultiHeadAttention(num_heads, model_dim, dropout)
        self.add_norm1 = AddNorm(model_dim, dropout)
        self.ffn = PositionwiseFFN(ffn_hidden_dim, model_dim)
        self.add_norm2 = AddNorm(model_dim, dropout)

    def forward(self, X):
        attention = self.attention(X, X, X)
        resid_out = self.add_norm1(X, attention)

        ffn_out = self.ffn(resid_out)
        return self.add_norm2(resid_out, ffn_out)

In [282]:
class RalphGPT(nn.Module):

    def __init__(
        self,
        num_blocks: int,
        num_heads: int,
        model_dim: int,
        ffn_hidden_dim: int,
        vocab_size: int,
        context_len: int,
        dropout = 0.5
    ):
        super().__init__()

        self.context_len = context_len
        self.model_dim = model_dim

        # TODO do we need a max_norm? seems like this would be important
        # depending on positional embedding scheme
        self.token_embedder = nn.Embedding(
            num_embeddings = vocab_size,
            embedding_dim = model_dim
        )

        # use learned positional embeddings for simplicity
        # TODO what are the tradeoffs with fixed positional embeddings besides less storage?
        self.positional_embedder = nn.Embedding(
            num_embeddings = context_len,
            embedding_dim = model_dim
        )

        self.decoder = nn.Sequential(*[
            TransformerDecoderBlock(num_heads, model_dim, ffn_hidden_dim, dropout)
            for _ in range(num_blocks)
        ])

        self.output = nn.LazyLinear(vocab_size)

    def forward(self, tokens):
        # tokens.shape is (batch size, ctx len)
        assert tokens.shape[-1] == self.context_len

        context_idx_tensor = torch.tensor(list(range(self.context_len)))
        positional_embeddings = self.positional_embedder(context_idx_tensor)

        token_embeddings = embedder(tokens)
        embeddings = token_embeddings + positional_embeddings

        return self.output(self.decoder(embeddings))


In [318]:
ralphgpt = RalphGPT(
    num_blocks = 10,
    num_heads = 4,
    model_dim = EMBEDDING_DIM,
    ffn_hidden_dim = EMBEDDING_DIM,
    vocab_size = tokenizer.n_vocab,
    context_len = CONTEXT_LEN
)

In [319]:
batches_x = torch.split(token_partitions_X, BATCH_SIZE, dim = 0)
batches_y = torch.split(token_partitions_Y, BATCH_SIZE, dim = 0)

In [320]:
loss_fn = nn.CrossEntropyLoss()
opt = torch.optim.SGD(ralphgpt.parameters(), lr=0.003)

In [323]:
num_epochs = 10

In [ ]:
for epoch in range(num_epochs):
    batches = list(zip(batches_x, batches_y))
    random.shuffle(batches)

    for batch_i, batch in enumerate(batches):
        batch_x, batch_y = batch
        pred_y = ralphgpt(batch_x)

        flat_pred_y = torch.flatten(pred_y, 0, 1)
        flat_batch_y = torch.flatten(batch_y, 0, 1)

        loss = loss_fn(flat_pred_y, flat_batch_y)
        print(f"[epoch: {epoch}][batch: {batch_i}] loss", loss.item())

        loss.backward()
        opt.step()
        opt.zero_grad()
        # val_pred_y = self.model(val_X)
        # val_loss = self.loss_fn(val_pred_y, val_y)

In [ ]:
def train(self, plot_cadence):
    train_dataloader, val_dataloader = self.dataset.get_dataloaders(self.hyperparams.general["batch_size"])

    print(train_dataloader)
    train_loss_hist = []
    val_loss_hist = []
    for epoch_num in range(self.hyperparams.general["num_epochs"]):
        for batch_num, (train_data, val_data) in enumerate(zip(train_dataloader, val_dataloader)):
            train_X, train_y = train_data
            val_X, val_y = val_data

            pred_y = self.model(train_X)
            loss = self.loss_fn(pred_y, train_y)
            print("loss", pred_y.float().mean(), train_y.float().mean(), loss)

            loss.backward()
            self.opt.step()
            self.opt.zero_grad()

            val_pred_y = self.model(val_X)
            val_loss = self.loss_fn(val_pred_y, val_y)
            
            if batch_num % plot_cadence == 0:
                train_loss_hist.append(loss.item())
                val_loss_hist.append(val_loss.item())

    return pd.DataFrame(dict(train=train_loss_hist, val=val_loss_hist))